<a href="https://colab.research.google.com/github/Sagargomad/My_projects/blob/main/Neural_Network_from_scratch_using_only_python%2C_without_a_deep_learning_library_like_TensorFlow_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

# Loading and preprocessing the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Defining the sigmoid activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x):
    return x * (1 - x)

# Creating a class for the two-layer neural network
class TwoLayerNN:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize the weights and biases for both layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.weights_input_hidden = np.random.uniform(-1, 1, (input_size, hidden_size))
        self.bias_hidden = np.zeros((1, hidden_size))

        self.weights_hidden_output = np.random.uniform(-1, 1, (hidden_size, output_size))
        self.bias_output = np.zeros((1, output_size))

    def forward(self, X):
        # Forward pass through the network
        self.input_layer = X
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = sigmoid(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output_layer_output = sigmoid(self.output_layer_input)
        return self.output_layer_output

    def backward(self, X, y, learning_rate):
        # Backpropagation
        error = y - self.output_layer_output

        # Computing gradients for the output layer
        delta_output = error * sigmoid_derivative(self.output_layer_output)
        d_weights_hidden_output = np.dot(self.hidden_layer_output.T, delta_output)
        d_bias_output = np.sum(delta_output, axis=0, keepdims=True)

        # Computing gradients for the hidden layer
        delta_hidden = np.dot(delta_output, self.weights_hidden_output.T) * sigmoid_derivative(self.hidden_layer_output)
        d_weights_input_hidden = np.dot(X.T, delta_hidden)
        d_bias_hidden = np.sum(delta_hidden, axis=0, keepdims=True)

        # Updating weights and biases
        self.weights_hidden_output += learning_rate * d_weights_hidden_output
        self.bias_output += learning_rate * d_bias_output
        self.weights_input_hidden += learning_rate * d_weights_input_hidden
        self.bias_hidden += learning_rate * d_bias_hidden

    def train(self, X, y, learning_rate, epochs):
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, learning_rate)

    def predict(self, X):
        return self.forward(X)

if __name__ == "__main__":
    # Defining network parameters
    input_size = 784  # 28x28
    hidden_size = 128
    output_size = 10

    # Initialize and train the neural network
    neural_net = TwoLayerNN(input_size, hidden_size, output_size)
    neural_net.train(train_images, train_labels, learning_rate=0.1, epochs=10)

    # Make predictions
    predictions = neural_net.predict(test_images)

    # Evaluate the accuracy
    accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(test_labels, axis=1))
    print("Accuracy:", accuracy)



Accuracy: 0.1028


# **for better accuracy  hyperparameter tuning is done**

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

# Loading and preprocessing the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Defining the ReLU activation function and its derivative
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Creating a class for the two-layer neural network
class TwoLayerNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initializing weights and biases with He initialization
        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * np.sqrt(2 / input_size)
        self.bias_hidden = np.zeros((1, hidden_size))

        self.weights_hidden_output = np.random.randn(hidden_size, output_size) * np.sqrt(2 / hidden_size)
        self.bias_output = np.zeros((1, output_size))

    def forward(self, X):
        self.input_layer = X
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = relu(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output_layer_output = np.exp(self.output_layer_input - np.max(self.output_layer_input, axis=1, keepdims=True))
        self.output_layer_output /= np.sum(self.output_layer_output, axis=1, keepdims=True)
        return self.output_layer_output

    def backward(self, X, y, learning_rate):
        m = X.shape[0]
        error = self.output_layer_output - y

        d_weights_hidden_output = np.dot(self.hidden_layer_output.T, error) / m
        d_bias_output = np.sum(error, axis=0, keepdims=True) / m

        delta_hidden = np.dot(error, self.weights_hidden_output.T) * relu_derivative(self.hidden_layer_input)
        d_weights_input_hidden = np.dot(X.T, delta_hidden) / m
        d_bias_hidden = np.sum(delta_hidden, axis=0, keepdims=True) / m

        self.weights_hidden_output -= learning_rate * d_weights_hidden_output
        self.bias_output -= learning_rate * d_bias_output
        self.weights_input_hidden -= learning_rate * d_weights_input_hidden
        self.bias_hidden -= learning_rate * d_bias_hidden

    def train(self, X, y, learning_rate, epochs, batch_size):
        for epoch in range(epochs):
            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i:i+batch_size]
                y_batch = y[i:i+batch_size]
                output = self.forward(X_batch)
                self.backward(X_batch, y_batch, learning_rate)

    def predict(self, X):
        return self.forward(X)

if __name__ == "__main__":
    # Defining network parameters
    input_size = 784  # 28x28
    hidden_size = 128
    output_size = 10

    # Initialize and train the neural network
    neural_net = TwoLayerNN(input_size, hidden_size, output_size)
    neural_net.train(train_images, train_labels, learning_rate=0.001, epochs=10, batch_size=64)

    # Make predictions
    predictions = neural_net.predict(test_images)

    # Evaluate the accuracy
    accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(test_labels, axis=1))
    print("Accuracy:", accuracy)


Accuracy: 0.8856
